In [1]:
# Import necessary libraries:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

The task is, given a set of five products produced by seven machines - each with their own efficiencies and maximal usage - produce the mix of products maximizing profit.

The first model solves this problem for a single-period. The second model introduces multiple periods and the ability to store products between periods, which differ in machine availability and demand per product.

In [2]:
# Single period model
m_j = gp.Model("ProductMixProblem")
m_j.ModelSense = GRB.MAXIMIZE
m_j.setParam('TimeLimit', 1000)

# Initialize decision variables, representing products and expected profit per product produced

## TorqueMaster
TM = m_j.addVar(vtype=GRB.CONTINUOUS, name="TorqueMaster", lb=0, obj=10)
## CryoLock
CL = m_j.addVar(vtype=GRB.CONTINUOUS, name="CryoLock", lb=0, obj=6)
## Axial-Flex
AF = m_j.addVar(vtype=GRB.CONTINUOUS, name="Axial-Flex", lb=0, obj=8)
## HydroForge
HF = m_j.addVar(vtype=GRB.CONTINUOUS, name="HydroForge", lb=0, obj=4)
## ThermaCore
TC = m_j.addVar(vtype=GRB.CONTINUOUS, name="ThermaCore", lb=0, obj=11)
# MagnoRail
MR = m_j.addVar(vtype=GRB.CONTINUOUS, name="MagnoRail", lb=0, obj=9)
## VibeShield
VS = m_j.addVar(vtype=GRB.CONTINUOUS, name="VibeShield", lb=0, obj=3)

# Initialize demand constraints for January as upper limits per product.
# Note that I am explicitly adding these as opposed to setting a ub parameter in the decision variables
# so that they are easier to adjust and analyze.

## TorqueMaster
m_j.addConstr(TM <= 500, name="TorqueMaster Demand")
## CryoLock
m_j.addConstr(CL <= 1000, name="CryoLock Demand")
## Axial-Flex
m_j.addConstr(AF <= 300, name="Axial-Flex Demand")
## HydroForge
m_j.addConstr(HF <= 300, name="HydroForge Demand")
## ThermaCore
m_j.addConstr(TC <= 800, name="ThermaCore Demand")
## MagnoRail
m_j.addConstr(MR <= 200, name="MagnoRail Demand")
## VibeShield
m_j.addConstr(VS <= 100, name="VibeShield Demand")

# The below expressions encode the amount of time required of each machine to produce one of each given product. nX refers to the number of available machines in January.

## Grinding
G = (0.5*TM + 0.7*CL + 0*AF + 0*HF + 0.3*TC + 0.2*MR + 0.5*VS)
nG = 3
## Vertical Drilling
VD = (0.1*TM + 0.2*CL + 0*AF + 0.3*HF + 0*TC + 0.6*MR + 0*VS)
nVD = 2
## Horizontal Drilling
HD = (0.2*TM + 0*CL + 0.8*AF + 0*HF + 0*TC + 0*MR + 0.6*VS)
nHD = 3
## Boring
B = (0.05*TM + 0.03*CL + 0*AF + 0.07*HF + 0.1*TC + 0*MR + 0.08*VS)
nB = 1
## Planing
P = (0*TM + 0*CL + 0.01*AF + 0*HF + 0.05*TC + 0*MR + 0.05*VS)
nP = 1

# The total time constraint is the number of working hours across 24 days.
hours = 16
working_days = 24
LC = hours * working_days

# Initialize time constraint as the amount each machine can be utilized, adjusted by the number of available machines.
m_j.addConstr(G <= LC * nG, name="G Time Constraint")
m_j.addConstr(VD <= LC * nVD, name="VD Time Constraint")
m_j.addConstr(HD <= LC * nHD, name="HD Time Constraint")
m_j.addConstr(B <= LC * nB, name="B Time Constraint")
m_j.addConstr(P <= LC * nP, name="P Time Constraint")

# Run model and display the optimal mix of products.
m_j.update()
m_j.display()
m_j.optimize()

[print(f'Optimal # of {var.VarName}s: {var.X}') for var in m_j.getVars()]
print(f'Optimal profit: ${m_j.ObjVal:0.2f}')


Set parameter Username
Set parameter LicenseID to value 2710302
Academic license - for non-commercial use only - expires 2026-09-18
Set parameter TimeLimit to value 1000
Maximize
10.0 TorqueMaster + 6.0 CryoLock + 8.0 Axial-Flex + 4.0 HydroForge + 11.0 ThermaCore
+ 9.0 MagnoRail + 3.0 VibeShield
Subject To
  TorqueMaster Demand: TorqueMaster <= 500
  CryoLock Demand: CryoLock <= 1000
  Axial-Flex Demand: Axial-Flex <= 300
  HydroForge Demand: HydroForge <= 300
  ThermaCore Demand: ThermaCore <= 800
  MagnoRail Demand: MagnoRail <= 200
  VibeShield Demand: VibeShield <= 100
G Time Constraint: 0.5 TorqueMaster + 0.7 CryoLock + 0.3 ThermaCore + 0.2 MagnoRail +
 0.5 VibeShield <= 1152
VD Time Constraint: 0.1 TorqueMaster + 0.2 CryoLock + 0.3 HydroForge + 0.6 MagnoRail <=
 768
  HD Time Constraint: 0.2 TorqueMaster + 0.8 Axial-Flex + 0.6 VibeShield <= 1152
B Time Constraint: 0.05 TorqueMaster + 0.03 CryoLock + 0.07 HydroForge + 0.1 ThermaCore
 + 0.08 VibeShield <= 384
  P Time Constraint: 0

C:\Users\harri\AppData\Local\Temp\ipykernel_26228\467124266.py:74: DeprecationWarning: Model.display() is deprecated
  m_j.display()


In [3]:
# January sensitivity analysis
# for var in m_j.getVars():
#     print(f'Variable: {var.varName}, Optimal Value = {var.x}, (LB,UB) = ({var.lb}, {var.ub}), Reduced cost = {var.RC}, Coeff = {var.obj}, Obj coeff range = ({var.SAObjLow: .3f}, {var.SAObjUp: .3f})')
    
for c in m_j.getConstrs():
    print(f'Constraint: {c.ConstrName}, RHS = {c.RHS}, slack = {c.slack}, Limits = ({c.SARHSLow}, {c.SARHSUp}, Shadow Price = {(c.Pi)})')
    
# By running a sensitivity analysis, I can see that the only binding constraint is on grinding time constraint. That constraint has a positive shadow
# price, so I would like more grinders. Through model testing, I find that I only need to replace the downed grinder to maximize demand.

Constraint: TorqueMaster Demand, RHS = 500.0, slack = 0.0, Limits = (344.0, 1744.0, Shadow Price = 5.714285714285714)
Constraint: CryoLock Demand, RHS = 1000.0, slack = 111.42857142857144, Limits = (888.5714285714286, inf, Shadow Price = 0.0)
Constraint: Axial-Flex Demand, RHS = 300.0, slack = 0.0, Limits = (0.0, 1315.0, Shadow Price = 8.0)
Constraint: HydroForge Demand, RHS = 300.0, slack = 0.0, Limits = (0.0, 1400.9523809523807, Shadow Price = 4.0)
Constraint: ThermaCore Demand, RHS = 800.0, slack = 0.0, Limits = (540.0, 2873.3333333333335, Shadow Price = 8.428571428571429)
Constraint: MagnoRail Demand, RHS = 200.0, slack = 0.0, Limits = (0.0, 808.4210526315788, Shadow Price = 7.285714285714286)
Constraint: VibeShield Demand, RHS = 100.0, slack = 100.0, Limits = (0.0, inf, Shadow Price = 0.0)
Constraint: G Time Constraint, RHS = 1152.0, slack = 0.0, Limits = (530.0, 1230.0, Shadow Price = 8.571428571428571)
Constraint: VD Time Constraint, RHS = 768.0, slack = 330.2857142857142, Limit

In [4]:
# Multi-period model

# Let's define our data as vectors and matrices.

# Iterators; We will use these later to build variables and constraints with loops.
products = ["TorqueMaster", "CryoLock", "Axial-Flex", "HydroForge", "ThermaCore", "MagnoRail", "VibeShield"]
months = ["January", "February", "March", "April", "May", "June"]
machines = ["Grinder", "Vertical Drill", "Horizontal Drill", "Borer", "Planer"]

# Used later for time constraints.
time_per_machine = 16 * 24

# Objective coefficients; We wil assign these to our products to form the objective function.
obj_coeffs = [10,6,8,4,11,9,3]

# Matrices

## The usage matrix encodes the amount of time required per machine per product, mapped across columns.
usage = np.array([
    [0.5,0.7,0,0,0.3,0.2,0.5], #Grinding
    [0.1,0.2,0,0.3,0,0.6,0], #Vertical Drilling 
    [0.2,0,0.8,0,0,0,0.6], #Horizontal Drilling
    [0.05,0.03,0,0.07,0.1,0,0.08], #Boring
    [0,0,0.01,0,0.05,0,0.05] #Planing
]) 

## The availability matrix displays the number of available machines as the total owned amount minus the predicted amount down for maintenance.

availability = np.array([
    [4-1, 2-0, 3-0, 1-0, 1-0], # January (1 Grinder down)
    [4-0, 2-0, 3-2, 1-0, 1-0], # February (2 Horiz. Drills down)
    [4-0, 2-0, 3-0, 1-0, 1-0], # March
    [4-0, 2-1, 3-0, 1-0, 1-0], # April (1 Vert. Drill down)
    [4-1, 2-1, 3-0, 1-0, 1-0], # May (1 Grinder, 1 Vert. Drill down)
    [4-0, 2-0, 3-1, 1-0, 1-0]  # June (1 Horiz. Drill down)
])

## Monthly capacity per machine type is calculated as the general time constraint multiplied by the number of available machines.

monthly_capacity = availability * time_per_machine

## The demand matrix encodes the demand per product per month.

demand = np.array([
    [500,1000,300,300,800,200,100], #January
    [600,500,200,0,400,300,150], #February
    [300,600,0,0,500,400,100], #March
    [200,300,400,500,200,0,100], #April
    [0,100,500,100,1000,300,0], #May
    [500,500,100,300,1100,500,60] #June
])

## The flag matrix is a convenient way to prevent the model from utili
flag = np.array([
    [0,0,0,0,0,0,0], #January
    [0,0,0,0,0,0,0], #Febraury
    #[0,0,0,0,0,0,0], #March if we had two borers 
    [1,1,0,1,1,0,1], #March
    [0,0,0,0,0,0,0], #April
    [0,0,0,0,0,0,0], #May
    [0,0,1,0,1,0,1] #June
    #[0,0,0,0,0,0,0] #June if we had two planers
])

In [5]:
# Create model
m_multi = gp.Model("Multiperiod Model")
m_multi.ModelSense = GRB.MAXIMIZE

# Decision variables: Products with objective coefficients
pvar_m = []
[pvar_m.append([m_multi.addVar(name = f'{products[i]}_{months[m]}', lb=0.0, obj=obj_coeffs[i]) for i in range(len(products))]) for m in range(len(months))]
m_multi.update()

# Decision variables: Storage with boundaries, which differ for the -1 month (initialization) and June, and have a cost of -0.50 for all intermediary months
# The initialization 'month', storage_init, prevents the model from using June storage to fulfill January's demand.
pvar_s = []
pvar_s.append([m_multi.addVar(name = f'{products[i]}_storage_init', lb=0.0, ub=0.0) for i in range(len(products))])
[pvar_s.append([m_multi.addVar(name = f'{products[i]}_{months[m]}_storage', lb=0.0, ub=100.0, obj=-0.50) for i in range(len(products))]) for m in range(len(months)-1)]
pvar_s.append([m_multi.addVar(name = f'{products[i]}_{months[-1]}_storage', lb=50.0, ub=50.0) for i in range(len(products))])
m_multi.update()

# Prevent sales beyond demand, prevent production when machines are down, and allow storage
for m in range(len(months)):
    for i in range(len(products)):
        sales_this_month = pvar_s[m][i] + pvar_m[m][i] - pvar_s[m+1][i]
        # This sales amount must be >= 0 and <= demand
        m_multi.addConstr(sales_this_month <= demand[m][i], name=f"Demand_{m}_{i}")
        m_multi.addConstr(sales_this_month >= 0, name=f"NoNegativeSales_{m}_{i}")
        
        if flag[m][i] == 1:
            m_multi.addConstr(pvar_m[m][i] == 0, name=f"Flagged_Month_{m}_Prod_{i}")

# Machine usage and time constraints, split across production for sales vs storage. This forces the model to consider overall production limits for both
# products produced for sales and products produced for storage.
for m in range(len(months)):
    for i in range(len(machines)):
        m_multi.addConstr(gp.quicksum(usage[i][p] * pvar_m[m][p] for p in range(len(products))) <= monthly_capacity[m][i])

m_multi.update()
m_multi.display()
m_multi.optimize()

# Print results
for m in range(len(months)):
    print(f'Production Schedule + Sold Storage for {months[m]}: ')
    for var in pvar_m[m]:
        print(f'    {var.VarName}: {var.X}')
    print(f'Storage Schedule for {months[m]}: ')
    for var in pvar_s[m+1]:
        print(f'    {var.VarName}: {var.X:0.2f}')
        
print(f'Optimal profit: ${m_multi.ObjVal:0.2f}')

Maximize
10.0 TorqueMaster_January + 6.0 CryoLock_January + 8.0 Axial-Flex_January
+ 4.0 HydroForge_January + 11.0 ThermaCore_January + 9.0 MagnoRail_January
+ 3.0 VibeShield_January + 10.0 TorqueMaster_February + 6.0 CryoLock_February + 8.0 Axial
-Flex_February + 4.0 HydroForge_February + 11.0 ThermaCore_February
+ 9.0 MagnoRail_February + 3.0 VibeShield_February + 10.0 TorqueMaster_March
+ 6.0 CryoLock_March + 8.0 Axial-Flex_March + 4.0 HydroForge_March
+ 11.0 ThermaCore_March + 9.0 MagnoRail_March + 3.0 VibeShield_March
+ 10.0 TorqueMaster_April + 6.0 CryoLock_April + 8.0 Axial-Flex_April
+ 4.0 HydroForge_April + 11.0 ThermaCore_April + 9.0 MagnoRail_April
+ 3.0 VibeShield_April + 10.0 TorqueMaster_May + 6.0 CryoLock_May + 8.0 Axial-Flex_May
+ 4.0 HydroForge_May + 11.0 ThermaCore_May + 9.0 MagnoRail_May + 3.0 VibeShield_May
+ 10.0 TorqueMaster_June + 6.0 CryoLock_June + 8.0 Axial-Flex_June + 4.0 HydroForge_June
+ 11.0 ThermaCore_June + 9.0 MagnoRail_June + 3.0 VibeShield_June +
-0.5

C:\Users\harri\AppData\Local\Temp\ipykernel_26228\1358605966.py:36: DeprecationWarning: Model.display() is deprecated
  m_multi.display()


Optimal objective  9.644017857e+04
Production Schedule + Sold Storage for January: 
    TorqueMaster_January: 500.0
    CryoLock_January: 888.5714285714287
    Axial-Flex_January: 382.5
    HydroForge_January: 300.0
    ThermaCore_January: 800.0
    MagnoRail_January: 200.0
    VibeShield_January: 0.0
Storage Schedule for January: 
    TorqueMaster_January_storage: 0.00
    CryoLock_January_storage: 0.00
    Axial-Flex_January_storage: 82.50
    HydroForge_January_storage: 0.00
    ThermaCore_January_storage: 0.00
    MagnoRail_January_storage: 0.00
    VibeShield_January_storage: 0.00
Production Schedule + Sold Storage for February: 
    TorqueMaster_February: 700.0
    CryoLock_February: 600.0
    Axial-Flex_February: 117.5
    HydroForge_February: 0.0
    ThermaCore_February: 500.0
    MagnoRail_February: 300.0
    VibeShield_February: 250.0
Storage Schedule for February: 
    TorqueMaster_February_storage: 100.00
    CryoLock_February_storage: 100.00
    Axial-Flex_February_storage